In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(500)

In [9]:
Corpus = pd.read_csv("corpus.csv")

In [12]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [13]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [14]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [16]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [17]:
print(Tfidf_vect.vocabulary_)

{'stun': 322, 'even': 96, 'sound': 308, 'track': 346, 'beautiful': 21, 'paint': 237, 'senery': 295, 'mind': 208, 'well': 369, 'would': 383, 'recomend': 272, 'people': 241, 'hate': 152, 'video': 364, 'game': 131, 'music': 217, 'play': 248, 'chrono': 42, 'cross': 62, 'ever': 98, 'best': 26, 'back': 19, 'away': 18, 'crude': 63, 'keyboarding': 181, 'take': 329, 'fresh': 125, 'step': 319, 'grate': 144, 'guitar': 149, 'soulful': 307, 'orchestra': 230, 'impress': 167, 'anyone': 11, 'care': 39, 'listen': 193, 'soundtrack': 309, 'anything': 12, 'read': 267, 'lot': 198, 'review': 281, 'say': 288, 'figure': 114, 'write': 384, 'disagree': 72, 'bit': 29, 'opinino': 228, 'yasunori': 387, 'mitsuda': 211, 'ultimate': 355, 'masterpiece': 204, 'timeless': 340, 'year': 388, 'beauty': 23, 'simply': 300, 'refuse': 274, 'price': 255, 'tag': 328, 'pretty': 254, 'stagger': 315, 'must': 218, 'go': 139, 'buy': 35, 'cd': 40, 'much': 216, 'money': 213, 'one': 227, 'feel': 111, 'worth': 382, 'every': 99, 'penny': 

In [18]:
print(Train_X_Tfidf)

  (0, 385)	0.15181783245925792
  (0, 373)	0.13182822297400662
  (0, 371)	0.13182822297400662
  (0, 339)	0.08347290620977138
  (0, 336)	0.15181783245925792
  (0, 316)	0.08347290620977138
  (0, 313)	0.15181783245925792
  (0, 299)	0.15181783245925792
  (0, 287)	0.13182822297400662
  (0, 283)	0.26365644594801324
  (0, 268)	0.15181783245925792
  (0, 267)	0.29296727954779
  (0, 253)	0.15181783245925792
  (0, 232)	0.15181783245925792
  (0, 227)	0.08347290620977138
  (0, 222)	0.13182822297400662
  (0, 218)	0.11764536933451467
  (0, 201)	0.15181783245925792
  (0, 200)	0.13182822297400662
  (0, 189)	0.15181783245925792
  (0, 180)	0.15181783245925792
  (0, 173)	0.30363566491851585
  (0, 156)	0.11764536933451467
  (0, 119)	0.15181783245925792
  (0, 117)	0.15181783245925792
  :	:
  (9, 279)	0.1256205421490419
  (9, 273)	0.1446688576328972
  (9, 261)	0.1446688576328972
  (9, 252)	0.1446688576328972
  (9, 233)	0.1446688576328972
  (9, 229)	0.1446688576328972
  (9, 220)	0.11210554723202179
  (9, 191)	

In [19]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  60.0


In [20]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  60.0
